In [14]:
from sqlalchemy import create_engine
import pandas as pd

user = 'jupyter_user'
password = '666-Zelda'
host = 'localhost'
port = '3306'
database = 'football'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [15]:
engine.connect()

In [16]:
with open('Football.sql', 'r', encoding='utf-8') as f:
    sql_script = f.read()

import mysql.connector

conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()
for statement in sql_script.split(';'):
    if statement.strip():
        cursor.execute(statement)
conn.commit()
cursor.close()
conn.close()

In [4]:
df = pd.read_sql("SELECT * FROM players", engine)
df

,player_id,team_id,name,position,date_of_birth,nationality
0,2,264,Juanmi,Offence,1993-05-20,Spain
1,8,90,Willian Jose,Offence,1991-11-23,Brazil
2,9,100,Diego Llorente,Defence,1993-08-16,Spain
3,11,92,Alvaro Odriozola,Defence,1995-12-14,Spain
4,14,586,Antonio Sanabria,Offence,1996-03-04,Paraguay
...,...,...,...,...,...,...
3145,252916,455,Luca Boncori,Offence,2006-02-21,Italy
3146,252918,450,Junior Ajayi,Offence,2004-10-11,Ivory Coast
3147,252993,558,Yoel Lago,Midfield,2004-03-25,Spain
3148,253001,558,Javi Rueda,Defence,2002-05-08,Spain


In [5]:
player_nationality = pd.read_sql('SELECT * FROM players WHERE nationality = "Scotland"', engine)
player_nationality

,player_id,team_id,name,position,date_of_birth,nationality
0,4359,356,Oliver McBurnie,Offence,1996-06-04,Scotland
1,5552,389,Jacob Brown,Offence,1998-04-10,Scotland
2,7846,55,Fraser Hornby,Midfield,1999-09-13,Scotland
3,7868,64,Andrew Robertson,Defence,1994-03-11,Scotland
4,7905,66,Scott McTominay,Midfield,1996-12-08,Scotland
5,7930,67,Matt Ritchie,Defence,1989-09-10,Scotland
6,15615,92,Kieran Tierney,Defence,1997-06-05,Scotland
7,34495,1044,Ryan Christie,Midfield,1995-02-22,Scotland
8,34638,103,Lewis Ferguson,Midfield,1999-08-24,Scotland
9,34646,58,John McGinn,Midfield,1994-10-18,Scotland


In [6]:
player_by_team = pd.read_sql("SELECT p.name,p.position,p.nationality,p.date_of_birth,t.name AS team_name FROM players p LEFT JOIN teams t ON p.team_id = t.team_id ORDER BY t.name", engine)
player_by_team

,name,position,nationality,date_of_birth,team_name
0,Seedy Jarju,Defence,Gambia,2004-10-28,1. FC Heidenheim 1846
1,Luka Janes,Midfield,Germany,2004-01-19,1. FC Heidenheim 1846
2,Elidon Qenaj,Offence,Germany,2003-05-22,1. FC Heidenheim 1846
3,Christopher Negele,Offence,Germany,2005-04-11,1. FC Heidenheim 1846
4,Adrian Beck,Midfield,Germany,1997-06-09,1. FC Heidenheim 1846
...,...,...,...,...,...
3145,Craig Dawson,Defence,England,1990-05-06,Wolverhampton Wanderers
3146,Thomas King,Goalkeeper,Wales,1995-03-09,Wolverhampton Wanderers
3147,Pedro Neto,Offence,Portugal,2000-03-09,Wolverhampton Wanderers
3148,Hwang Heechan,Offence,South Korea,1996-01-26,Wolverhampton Wanderers


In [7]:
df['nationality'].value_counts()

nationality
Spain         506
France        402
Germany       319
England       300
Italy         299
             ... 
Andorra         1
Burundi         1
Panama          1
Honduras        1
Luxembourg      1
Name: count, Length: 107, dtype: int64

In [25]:
immigrants_league = pd.read_sql(
    """
    WITH player_data AS (
        SELECT 
            p.player_id,
            p.name AS player_name,
            p.nationality,
            t.team_id,
            t.name AS team_name,
            s.position AS team_ranking,
            l.league_id,
            l.name AS league_name,
            CASE
                WHEN l.name = 'Premier League' AND p.nationality IN ('England', 'Wales') THEN 0
                WHEN l.name = 'Serie A' AND p.nationality = 'Italy' THEN 0
                WHEN l.name = 'La Liga' AND p.nationality = 'Spain' THEN 0
                WHEN l.name = 'Bundesliga' AND p.nationality = 'Germany' THEN 0
                WHEN l.name = 'Ligue 1' AND p.nationality = 'France' THEN 0
                ELSE 1
            END AS is_immigrant
        FROM players p
        JOIN teams t ON p.team_id = t.team_id
        JOIN leagues l ON t.league_id = l.league_id
        JOIN standings s ON t.team_id = s.team_id
    ),
    league_stats AS (
        SELECT 
            league_name,
            COUNT(*) AS total_players,
            SUM(is_immigrant) AS immigrant_players,
            ROUND(100.0 * SUM(is_immigrant) / COUNT(*), 2) AS immigrant_percentage
        FROM player_data
        GROUP BY league_name
    )
    SELECT 
        league_name,
        total_players,
        immigrant_players,
        immigrant_percentage
    FROM league_stats
    ORDER BY immigrant_percentage DESC
    """,
    engine
)

immigrants_league

,league_name,total_players,immigrant_players,immigrant_percentage
0,Serie A,673,388.0,57.65
1,Premier League,636,361.0,56.76
2,Ligue 1,554,273.0,49.28
3,Bundesliga,555,262.0,47.21
4,La Liga,694,246.0,35.45


In [39]:
immigrants_team = pd.read_sql(
    """
    WITH player_data AS (
        SELECT 
            p.player_id,
            p.name AS player_name,
            p.nationality,
            t.team_id,
            t.name AS team_name,
            s.position AS team_ranking,
            l.league_id,
            l.name AS league_name,
            CASE
                WHEN l.name = 'Premier League' AND p.nationality IN ('England', 'Wales') THEN 0
                WHEN l.name = 'Serie A' AND p.nationality = 'Italy' THEN 0
                WHEN l.name = 'La Liga' AND p.nationality = 'Spain' THEN 0
                WHEN l.name = 'Bundesliga' AND p.nationality = 'Germany' THEN 0
                WHEN l.name = 'Ligue 1' AND p.nationality = 'France' THEN 0
                ELSE 1
            END AS is_immigrant
        FROM players p
        JOIN standings s ON s.team_id = p.team_id
        JOIN teams t ON p.team_id = t.team_id
        JOIN leagues l ON t.league_id = l.league_id
    ),
    nationality_counts AS (
        SELECT 
            team_name,
            league_name,
            nationality,
            COUNT(*) AS num_players
        FROM player_data
        GROUP BY team_name, league_name, nationality
    ),
    nationality_summary AS (
        SELECT 
            team_name,
            league_name,
            GROUP_CONCAT(CONCAT(nationality, ' (', num_players, ')') ORDER BY num_players DESC SEPARATOR ', ') AS player_nationality_breakdown
        FROM nationality_counts
        GROUP BY team_name, league_name
    ),
    team_stats AS (
        SELECT 
            league_name,
            team_name,
            team_ranking,
            COUNT(*) AS team_total_players,
            SUM(is_immigrant) AS team_immigrant_players,
            ROUND(100 * SUM(is_immigrant) / COUNT(*), 2) AS team_immigrant_percentage
        FROM player_data
        GROUP BY league_name, team_name, team_ranking
    )
    SELECT 
        ts.league_name,
        ts.team_name,
        ts.team_ranking,
        ts.team_total_players,
        ts.team_immigrant_players,
        ts.team_immigrant_percentage,
        ns.player_nationality_breakdown
    FROM team_stats ts
    JOIN nationality_summary ns
      ON ts.team_name = ns.team_name AND ts.league_name = ns.league_name
    ORDER BY ts.league_name, ts.team_immigrant_percentage DESC
    """,
    engine)

pd.set_option('display.max_rows', None)
immigrants_team.head(1000)

,league_name,team_name,team_ranking,team_total_players,team_immigrant_players,team_immigrant_percentage,player_nationality_breakdown
0,Bundesliga,Bayer 04 Leverkusen,1,29,20.0,68.97,"Germany (9), Czech Republic (3), Netherlands (..."
1,Bundesliga,RB Leipzig,4,27,18.0,66.67,"Germany (9), France (3), Austria (3), Netherla..."
2,Bundesliga,VfL Wolfsburg,12,29,18.0,62.07,"Germany (11), Belgium (3), Denmark (2), Sweden..."
3,Bundesliga,Eintracht Frankfurt,6,37,22.0,59.46,"Germany (15), France (4), Ecuador (2), Sweden ..."
4,Bundesliga,FC Bayern Munchen,3,35,20.0,57.14,"Germany (15), France (4), Morocco (2), England..."
5,Bundesliga,Borussia Monchengladbach,14,32,18.0,56.25,"Germany (14), France (5), USA (2), Switzerland..."
6,Bundesliga,1. FC Union Berlin,15,27,15.0,55.56,"Germany (12), Denmark (3), France (2), Tunisia..."
7,Bundesliga,SV Werder Bremen,9,30,15.0,50.00,"Germany (15), Austria (2), Belgium (2), France..."
8,Bundesliga,FC Augsburg,11,30,14.0,46.67,"Germany (16), Switzerland (2), Croatia (2), Ne..."
9,Bundesliga,VfL Bochum 1848,16,30,14.0,46.67,"Germany (16), Ghana (2), Brazil (2), Slovakia ..."
